importing tensorflow
- 每次运行时必须创建session

In [2]:
import tensorflow as tf
import os

#turn off tensorflow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# define d=computational graph
X=tf.placeholder(tf.float32,name="X")
Y=tf.placeholder(tf.float32,name="Y")

addition=tf.add(X,Y,name="addition")

#create the session
with tf.Session() as session:
    result=session.run(addition,feed_dict={X:[1,5,6],Y:[4,7,9]})
    print(result)

[  5.  12.  15.]


In [7]:
p=tf.constant(-1)
l=tf.constant(5)
resul2=tf.add(p,l,name='resul2')
print(resul2)

Tensor("resul2:0", shape=(), dtype=int32)


# 电子游戏销售预测

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

#make the data
# col=['critic_rating','is_action','is_exclusive_to_us','is_portable','is_role_playing','is_sequel','is_sports','suitable_for_kids','total_earnings','unit_price']
col=['critic_rating','is_action','is_exclusive_to_us','is_portable','is_role_playing','is_sequel','is_sports','suitable_for_kids','total_earnings','unit_price']
# trainData={} #store the trainData
# testData={}  #store the testData
# # prediceData={}#store the predict data
# for i in range(len(col)):
#     if i==0:
#         testData[col[i]]=np.random.uniform(1,5,10000)
#     elif i==(len(col)-1):
#         testData[col[i]]=np.random.uniform(30,60,10000)
#     elif  i==len(col)-2:
#         testData[col[i]]=np.random.randint(30000,140000,10000)
#     else:
#         testData[col[i]]=np.random.randint(0,2,10000)
        
# testData=pd.DataFrame(testData)  


In [3]:
#model.py

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
  

tranining_data_df=pd.read_excel("data/gameTranindata.xlsx",dtype=float)

X_training=tranining_data_df.drop('total_earnings',axis=1).values
Y_training=tranining_data_df[['total_earnings']].values


test_data_df=pd.read_excel("data/gameTestdata.xlsx",dtype=float)

X_testing=test_data_df.drop('total_earnings',axis=1).values
Y_testing=test_data_df[['total_earnings']].values

#all data needs to be scaled to a small range like 0 to 1 for the neural
X_scaler=MinMaxScaler(feature_range=(0,1))#使每个feaure值在0到1之间
Y_scaler=MinMaxScaler(feature_range=(0,1))

#sacle both the training inputs and outputs
X_scaled_training=X_scaler.fit_transform(X_training)
Y_scaled_training=Y_scaler.fit_transform(Y_training)

# 
X_scaled_testing=X_scaler.transform(X_testing)
Y_scaled_testing=Y_scaler.transform(Y_testing)

print(X_scaled_testing.shape)
print(Y_scaled_testing.shape)
print("note:Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(Y_scaler.scale_[0],Y_scaler.min_[0]))

(10000, 9)
(10000, 1)
note:Y values were scaled by multiplying by 0.0000090917 and adding -0.2728


In [4]:
#model.py
# define model parameters
learning_rate=0.001 #学习率
training_epochs=100 # 迭代次数
display_step=5 #每5次迭代展示一次

#define how many inputs and outputs are in our neural network
number_of_inputs=9
number_of_outputs=1

#define how many neurons we want in each layer of our neural network
layer_1_nodes=50
layer_2_nodes=100
layer_3_nodes=50


#section one: define the layers of the neural network itself

#input layer
with tf.variable_scope('input'):
    X=tf.placeholder(tf.float32,shape=(None,number_of_inputs))

#layer 1
with tf.variable_scope('layer_1'):
    weights=tf.get_variable(name="weights1",shape=[number_of_inputs,layer_1_nodes],initializer=tf.contrib.layers.xavier_initializer())
    biases=tf.get_variable(name="baiases1",shape=[layer_1_nodes],initializer=tf.zeros_initializer())
    layer_1_output=tf.nn.relu(tf.matmul(X,weights)+biases)

#layer 2
with tf.variable_scope('layer_2'):
    weights=tf.get_variable(name="weights2",shape=[layer_1_nodes,layer_2_nodes],initializer=tf.contrib.layers.xavier_initializer())
    biases=tf.get_variable(name="baiases2",shape=[layer_2_nodes],initializer=tf.zeros_initializer())
    layer_2_output=tf.nn.relu(tf.matmul(layer_1_output,weights)+biases)
    
#layer 3
with tf.variable_scope('layer_3'):
    weights=tf.get_variable(name="weights3",shape=[layer_2_nodes,layer_3_nodes],initializer=tf.contrib.layers.xavier_initializer())
    biases=tf.get_variable(name="baiases3",shape=[layer_3_nodes],initializer=tf.zeros_initializer())
    layer_3_output=tf.nn.relu(tf.matmul(layer_2_output,weights)+biases)


#output layer
with tf.variable_scope('output'):
    weights=tf.get_variable(name="weights4",shape=[layer_3_nodes,number_of_outputs],initializer=tf.contrib.layers.xavier_initializer())
    biases=tf.get_variable(name="baiases4",shape=[number_of_outputs],initializer=tf.zeros_initializer())
    prediction=tf.nn.relu(tf.matmul(layer_3_output,weights)+biases)

#section two: define the cost function of the neural network that will measure prediction

with tf.variable_scope('cost'):
    Y=tf.placeholder(tf.float32,shape=(None,1))
    cost=tf.reduce_mean(tf.squared_difference(prediction,Y))
    
#section three: define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)

#'''使用tensorboard 展示数据. 1、保持日志文件 2、使用tensorboard  图标展示日志文件 
#    3、程序执行完后,在终端执行:tensorboard 把生成的链接贴到浏览器中查看'''

# create a summary operation to log the progress of the network
# with tf.variable_scope('logging'):
#     tf.summary.scalar('current_cost',cost)
#     summary=tf.summary.merge_all()


Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
# training_loop.py 内容包括接上面model.py

#'''使用tensorboard 展示数据. 1、保持日志文件 2、使用tensorboard  图标展示日志文件 
#    3、程序执行完后,在终端执行:tensorboard 把生成的链接贴到浏览器中查看'''
# create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost',cost)
    summary=tf.summary.merge_all()
    
#initialize a session so that we can run tensorflow  operations
with tf.Session() as session:
    #run the global variable initializer to initialize all variables and layers of
    session.run(tf.global_variables_initializer())
    
    #create log file writers to record training progress.
    #we'll store training and testing log data separately
    training_writer=tf.summary.FileWriter("data/logs/training",session.graph)
    testing_writer=tf.summary.FileWriter("data/logs/testing",session.graph)
    
    
    #run the optimizer over and over to train the network
    #one epoch is one full run through the training data set.
    for epoch in range(training_epochs):
        
        #feed in the training data and do one step of neural network tranining
        session.run(optimizer,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
        
        #print the current training status to the screen
#         print("training pass:{}".format(epoch))
        
        #every 5 training steps,log our progress
        if epoch % 5 ==0:
            training_cost,training_summary=session.run([cost,summary],feed_dict={X:X_scaled_training,Y:Y_scaled_training})
            testing_cost,testing_summary=session.run([cost,summary],feed_dict={X:X_scaled_testing,Y:Y_scaled_testing})
            
            #write the current training staus to the log files(which we can visai)
            training_writer.add_summary(training_summary,epoch)
            testing_writer.add_summary(testing_summary,epoch)
            
            print(epoch,training_cost,testing_cost)
            
            
    #training is now complete!
    print("training is complete")
    
    #print the last traning_cost and testing_cost
    final_training_cost=session.run(cost,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
    final_testing_cost=session.run(cost,feed_dict={X:X_scaled_testing,Y:Y_scaled_testing})
    
    print("finanl traing cost:{}".format(final_training_cost))
    print("final testing cost:{}".format(final_testing_cost))   


0 0.270431 0.272129
5 0.100326 0.0996455
10 0.119588 0.11819
15 0.0933929 0.0927999
20 0.0971223 0.0967862
25 0.0921832 0.0916579
30 0.0893359 0.0886045
35 0.0896593 0.0889946
40 0.0875677 0.0871659
45 0.0876617 0.0874575
50 0.0867465 0.086623
55 0.0865773 0.0865123
60 0.0860007 0.0860662
65 0.0857365 0.085927
70 0.0853411 0.0856331
75 0.0850475 0.0854885
80 0.084719 0.085359
85 0.0844274 0.0852553
90 0.0841628 0.0851786
95 0.0839262 0.0851104
training is complete
finanl traing cost:0.08375921845436096
final testing cost:0.08507178723812103


In [ ]:
#使用tensorboard 展示数据. 1、保持日志文件 2、使用tensorboard  图标展示日志文件


In [43]:
import numpy as np
col=np.random.rand(0,2,5)
p=np.random.uniform(0,2,4)
print(p)


[ 1.19852506  1.71705731  1.10382514  1.52350191]


In [22]:
result=sess.run(c)

In [23]:
print(result)

5


In [24]:
sess.close()

# tensorflow tutorials--MNIST

In [2]:
# set up the skeleton for our tensorflwo program
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


#our application logic will be added here
if __name__=="__main__":
    tf.app.run()
      

AttributeError: module '__main__' has no attribute 'main'

In [2]:
def cnn_model_fn(features,labels,mode):
    """model function for cnn."""
    #input layer
    input_layer=tf.reshape(features["x"],[-1,28,28,1])
    
    #convolutional layer #1
    conv1=tf.layers.comv2d(
        inputs=input_layer,
        filters=32,  #?????什么意思?
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    
    #pooling layer #1
    pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],stides=2)#what are the means of arguments?
    
    #convolutional layer #2 and pooling layer #2
    conv2=tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5]
        padding="same",
        activation=tf.nn.relu)
    pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    
    #dense layer
    pool2_flat=tf.reshape(pool2,[-1,7*7*64])
    dense=tf.layers.dense(inputs=pool2_flat,units=1024,activation=tf.nn.relu)
    dropout=tf.layers.dropout(inputs=dense,rate=0.4,training=mode==tf.estimator.ModeKeys.TRAIN)
    
    #logits layer
    logits=tf.layers.dense(inputs=dropput,units=10)
    
    predictions={
        #generrate predictions(for PREDICT and EVAL mode)
        "class":tf.argmax(input=logits,axis=1)
        #add 'softmax_tensor' to the graph.it is used for PREDICT and by the  logging_hook
        "probabilities":tf.nn.softmax(logits,name="softmax_tensor")
    }
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    #calculate loss (for both train and eval modes)
    loss=tf.losses.spare_softmax_cross_entropy(labels=labels,logits=logits)
    
    #configure the training op(for train mode)
    if mode=tf.estimator.ModeKeys.TRAIN:
        optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op=optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    #add evaluation metrics(for eval mode)
    eval_metric_ops={"accuracy":tf.metrics.accuracy(labels=labels,predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    #load training and eval data
    mnist=tf.contrib.learn.datasets.load_dataset("mnist")
    train_data=minist.train.images#returns np.array
    train_labels=np.asarray(minist.train.labels,dtype=np.int32)
    eval_data=mnist.test.images#returns np.array
    eval_labels=np.asarray(mnist.test.labels,dtype=np.int32)
 

#create the estimator
mnist_classifier=tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="data/modes/mnist_convet_model")

#set up logging for predictions
tensors_to_log={"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

#train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])


#evaluate the model and print results
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


SyntaxError: invalid syntax (<ipython-input-2-4b0ef73f57db>, line 22)

In [1]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-

INFO:tensorflow:loss = 2.32599, step = 1
INFO:tensorflow:probabilities = [[ 0.10343225  0.10163384  0.11327754  0.10177872  0.0977136   0.09289385
   0.09844957  0.09769947  0.09884666  0.09427448]
 [ 0.10921008  0.10191244  0.10802146  0.08844501  0.1020833   0.0979012
   0.09656551  0.0928885   0.10596535  0.09700713]
 [ 0.09931476  0.09868395  0.1036617   0.09707054  0.10745269  0.09423973
   0.09069649  0.09370019  0.10882764  0.10635229]
 [ 0.09353085  0.1048191   0.11077613  0.0915764   0.10756812  0.10659175
   0.09401757  0.09307634  0.09419398  0.10384975]
 [ 0.1066525   0.09443699  0.1089616   0.10210783  0.0957534   0.0949548
   0.09893067  0.1042232   0.08653004  0.10744899]
 [ 0.10274138  0.10208233  0.11165892  0.09135567  0.10096519  0.09665567
   0.09696421  0.09040077  0.10470323  0.10247259]
 [ 0.10598424  0.09835371  0.12679373  0.08787375  0.0928613   0.08984052
   0.09344871  0.09176563  0.10039295  0.11268544]
 [ 0.0950854   0.0959961   0.11586347  0.09069628  0.0

INFO:tensorflow:global_step/sec: 5.8696
INFO:tensorflow:probabilities = [[ 0.10157643  0.1054237   0.10714445  0.09566553  0.10340902  0.0980543
   0.09337593  0.09829812  0.09639454  0.10065794]
 [ 0.10587794  0.0947562   0.11018158  0.10147193  0.10185402  0.10036322
   0.09432625  0.0934203   0.0983392   0.09940933]
 [ 0.09054609  0.10230929  0.11005872  0.10043325  0.10143561  0.09424527
   0.09363911  0.0964144   0.10336784  0.10755049]
 [ 0.0872244   0.10502327  0.11371048  0.09729776  0.11506118  0.09448692
   0.0935143   0.08613185  0.09646399  0.11108582]
 [ 0.1061219   0.1029824   0.11420835  0.08501456  0.09496055  0.10219406
   0.08873724  0.09602097  0.10674134  0.1030186 ]
 [ 0.09540038  0.10227685  0.10827131  0.10097679  0.11299372  0.09415659
   0.09842762  0.0944513   0.09798598  0.09505948]
 [ 0.09565299  0.10503683  0.1056259   0.10505375  0.11078946  0.08925487
   0.0893999   0.09853391  0.09997965  0.10067269]
 [ 0.08911046  0.11086448  0.11007453  0.08448023  0.1

INFO:tensorflow:loss = 2.29422, step = 101 (17.036 sec)
INFO:tensorflow:probabilities = [[ 0.10457115  0.10102173  0.10578125  0.09810439  0.10691517  0.09916902
   0.09839036  0.08647919  0.09831595  0.10125177]
 [ 0.09260833  0.10288856  0.09702152  0.09646959  0.11013936  0.0975787
   0.09930705  0.10043544  0.10128687  0.10226453]
 [ 0.11644691  0.10804043  0.12015725  0.09000594  0.09125751  0.08826548
   0.08559728  0.09017182  0.10089595  0.10916151]
 [ 0.09668802  0.10216969  0.11410182  0.10155963  0.10581165  0.08437549
   0.0992467   0.09130229  0.10465087  0.10009386]
 [ 0.09802097  0.09733626  0.09929586  0.09966894  0.11515626  0.10186052
   0.09123927  0.08696508  0.09936578  0.11109101]
 [ 0.09875348  0.10112293  0.10985623  0.09786775  0.09671221  0.09017112
   0.09185384  0.10224188  0.09672514  0.11469543]
 [ 0.09066436  0.10779022  0.1057232   0.0848306   0.11146002  0.09780335
   0.08879374  0.09230251  0.11052673  0.11010525]
 [ 0.10781655  0.09355585  0.1114461  

INFO:tensorflow:global_step/sec: 5.84505
INFO:tensorflow:probabilities = [[ 0.10053402  0.10093413  0.11290628  0.09885496  0.09830894  0.10351192
   0.09617208  0.09426062  0.09972733  0.09478965]
 [ 0.09433874  0.10146111  0.10237706  0.09206899  0.10012092  0.10488214
   0.09962501  0.0916751   0.11146678  0.10198424]
 [ 0.10669003  0.11356639  0.10834613  0.10198293  0.08963568  0.08849739
   0.09563091  0.10045911  0.10475715  0.09043428]
 [ 0.11389101  0.09551381  0.11911867  0.10172549  0.09597852  0.09532854
   0.08942487  0.09261001  0.09923043  0.09717876]
 [ 0.09801485  0.10277592  0.1164466   0.09324662  0.11040146  0.09406632
   0.08745269  0.09812928  0.09899319  0.10047308]
 [ 0.10376691  0.10743959  0.1094953   0.09477191  0.09228441  0.08701251
   0.10167976  0.10678519  0.09554602  0.10121837]
 [ 0.08652911  0.08697824  0.10987787  0.10437089  0.09784031  0.10961188
   0.10359372  0.09743109  0.11144389  0.09232304]
 [ 0.10658754  0.10953409  0.10684878  0.09147106  0

...

INFO:tensorflow:global_step/sec: 5.6734
INFO:tensorflow:probabilities = [[ 0.09582985  0.08795915  0.09830311  0.10100741  0.10104274  0.10909833
   0.09283397  0.10335453  0.10286114  0.10770978]
 [ 0.10428338  0.09052046  0.09970364  0.08705212  0.11326254  0.08527367
   0.10028768  0.08974183  0.1234653   0.1064094 ]
 [ 0.10631108  0.10567356  0.10846002  0.0961052   0.09681249  0.10084724
   0.09788214  0.09416576  0.09848024  0.09526229]
 [ 0.12020034  0.09277736  0.10224581  0.08999369  0.09744578  0.08570607
   0.10094098  0.09673054  0.11917464  0.09478484]
 [ 0.10416296  0.10010632  0.09955832  0.09962988  0.103236    0.0940593
   0.0952606   0.09735925  0.10069466  0.10593271]
 [ 0.09603655  0.08931705  0.1016034   0.10756029  0.10436949  0.09687504
   0.09143835  0.10407441  0.10537295  0.10335245]
 [ 0.09059211  0.1017735   0.10035861  0.09856886  0.10087072  0.09455733
   0.10201832  0.09952543  0.10447831  0.10725679]
 [ 0.10779995  0.105917    0.10812295  0.09475079  0.0

INFO:tensorflow:loss = 2.26059, step = 401 (17.627 sec)
INFO:tensorflow:probabilities = [[ 0.10940592  0.09193227  0.1049511   0.10281099  0.11415455  0.07907294
   0.08523299  0.08989301  0.11568087  0.10686531]
 [ 0.10311554  0.09985098  0.09875515  0.10354508  0.10684851  0.08510327
   0.10349617  0.09693368  0.09786624  0.10448536]
 [ 0.10169364  0.0990688   0.1069536   0.10290079  0.09931566  0.0899144
   0.10721134  0.0809075   0.09923963  0.11279467]
 [ 0.1042388   0.08020748  0.14707215  0.10117883  0.09729939  0.08536743
   0.08822282  0.09021088  0.111969    0.09423324]
 [ 0.10800438  0.09346939  0.11477197  0.08900634  0.09040853  0.09361637
   0.09504143  0.10307249  0.11248701  0.10012201]
 [ 0.11826206  0.08931812  0.10072858  0.10811847  0.09120985  0.09538344
   0.08267272  0.08684359  0.10604144  0.12142168]
 [ 0.09234492  0.09417005  0.11376958  0.11294535  0.09407111  0.08341525
   0.11439654  0.09392656  0.0983449   0.1026157 ]
 [ 0.08448651  0.09128818  0.10776328 

INFO:tensorflow:global_step/sec: 5.42112
INFO:tensorflow:probabilities = [[ 0.08981386  0.08738995  0.09394916  0.11301885  0.10630744  0.08910614
   0.10809409  0.10665477  0.10360356  0.10206211]
 [ 0.1015413   0.09714746  0.11046288  0.11053274  0.09276791  0.08939563
   0.08613151  0.10999762  0.1038971   0.09812587]
 [ 0.09460279  0.09870596  0.10881232  0.10985542  0.10269371  0.09544756
   0.10133152  0.09935673  0.10046914  0.08872489]
 [ 0.09174289  0.10241986  0.11305233  0.10169758  0.10208864  0.09334816
   0.09290493  0.09301712  0.10473032  0.10499802]
 [ 0.0982032   0.10660134  0.11110257  0.10325376  0.09839263  0.08795477
   0.09861344  0.09488598  0.09149483  0.10949748]
 [ 0.10747917  0.09730601  0.10463203  0.09506118  0.09772133  0.08342811
   0.09989933  0.10471979  0.10824817  0.10150488]
 [ 0.1162504   0.09404715  0.11612194  0.10507382  0.08519047  0.09860472
   0.0905885   0.09127557  0.1076365   0.09521089]
 [ 0.11249939  0.08885524  0.10590702  0.09107929  0

INFO:tensorflow:loss = 2.24367, step = 501 (18.447 sec)


KeyboardInterrupt: 

In [20]:
import tensorflow as tf
weights=tf.Variable(tf.random_normal([2,3],stddev=2))
weights

<tf.Variable 'Variable_6:0' shape=(2, 3) dtype=float32_ref>

In [21]:
tf.zeros([2,3])

<tf.Tensor 'zeros_3:0' shape=(2, 3) dtype=float32>

In [23]:
# biases=tf=tf.Variable(tf.zeros([3]))
w2=tf.Variable(weights.initialized_value())
w3=tf.Variable(weights.initialized_value()*2.0)

In [34]:
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
x=tf.constant([[0.7,0.9]])
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)
sess=tf.Session()
sess.run(w1.initializer)
sess.run(w2.initializer)
print(sess.run(y))
sess.close()

[[ 3.95757794]]


In [36]:
import tensorflow as tf
from numpy.random import RandomState

#定义训练数据batch 的大小
batch_size=8

# 定义神经网络的参数
w1=tf.Variable(tf.random_normal(([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))
               
x=tf.placeholder(tf.float32,shape=(None,2),name='x-input')      
y_=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

# define 神经网络前向传播的过程
a=tf.matnul(x,w1)
y=tf.matmul(a,w2)
               
#定义损失函数的反向传播过程和损失函数
cross_entropy=-tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,le-10,1.0)))
train_step=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

SyntaxError: invalid syntax (<ipython-input-36-eedad6d94d13>, line 8)

In [37]:
# rdm=RandomState(1)
# dateaset_size=128
# X=rdm.rand(dataset_size,2)
# Y=[[int(x1+x2<1) for (x1,x2) in X]]

In [39]:
import tensorflow as tf
# tf.enable_eager_excution()
print(tf.add(1,2))

Tensor("Add:0", shape=(), dtype=int32)


In [40]:
print(tf.add([1,2],[3,4]))

Tensor("Add_1:0", shape=(2,), dtype=int32)


In [41]:
print(tf.square(5))

Tensor("Square:0", shape=(), dtype=int32)


In [42]:
print(tf.reduce_sum([1,2,3]))
print(tf.encode_base64("hello world"))

Tensor("Sum:0", shape=(), dtype=int32)
Tensor("EncodeBase64:0", shape=(), dtype=string)


In [43]:
print(tf.square(2)+tf.square(3))

Tensor("add:0", shape=(), dtype=int32)
